In [58]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LinearRegression

In [59]:
# Define paths to the individual CSV files
csv_files = {
    '1': 'outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest-emissiondata-1-examples.csv',
    '5': 'outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest-emissiondata-5-examples.csv',
    '10': 'outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest-emissiondata-10-examples.csv',
    '30': 'outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest-emissiondata-30-examples.csv',
    #'90': 'outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest-emissiondata-90-examples.csv',
}

# Read the emissions data
emissions_data = pd.read_csv('emissions.csv')

In [60]:
# Initialize lists to store metadata
parameters = []
num_examples = []
num_prompts = []
total_emissions = []
cpu_energy = []
gpu_energy = []
ram_energy = []
total_output_tokens = []
total_input_tokens = []
avg_input_tokens = []
avg_output_tokens = []

In [61]:
# Read and extract metadata from each CSV file
for model, file in csv_files.items():
    data = pd.read_csv(file)
    prompts = data.loc[data['Metric'] == 'Total Prompts', 'Value'].values[0]
    output_tokens = data.loc[data['Metric'] == 'Total Output Tokens', 'Value'].values[0]
    input_tokens = data.loc[data['Metric'] == 'Total Input Tokens', 'Value'].values[0]
    avg_i_tok = data.loc[data['Metric'] == 'Avg Input Tokens per Prompt', 'Value'].values[0]
    avg_o_tok = data.loc[data['Metric'] == 'Avg Output Tokens per Prompt', 'Value'].values[0]
    parameters.append(70)
    num_examples.append(int(model))
    num_prompts.append(int(prompts))
    total_output_tokens.append(float(output_tokens))
    total_input_tokens.append(float(input_tokens))
    avg_input_tokens.append(float(avg_i_tok))
    avg_output_tokens.append(float(avg_o_tok))    

In [62]:
# Extract emissions data
for model in csv_files.keys():
    model_emissions = emissions_data[emissions_data['project_name'].str.contains("Llama-3-8B-Instruct-OutputTest-emissiondata-" + model + "-")]
    total_emissions.append(model_emissions['emissions'].values[0])
    cpu_energy.append(model_emissions['cpu_energy'].values[0])
    gpu_energy.append(model_emissions['gpu_energy'].values[0])
    ram_energy.append(model_emissions['ram_energy'].values[0])

In [63]:
print(avg_input_tokens)
print(avg_output_tokens)
print(total_output_tokens)
print(total_emissions)

[186.66, 186.66, 186.66, 186.66]
[18.027, 73.053, 176.913, 587.78]
[2704.0, 10958.0, 26537.0, 29389.0]
[0.0107639213548215, 0.0439809911016423, 0.1084166041887974, 0.1236233887845658]


In [64]:
# Prepare data for regression and visualization
parameters = np.array(parameters)
num_examples = np.array(num_examples)
num_prompts = np.array(num_prompts)
total_output_tokens = np.array(total_output_tokens)
total_input_tokens = np.array(total_input_tokens)
avg_input_tokens = np.array(avg_input_tokens)
avg_output_tokens = np.array(avg_output_tokens)
total_emissions = np.array(total_emissions)
cpu_energy = np.array(cpu_energy)
gpu_energy = np.array(gpu_energy)
ram_energy = np.array(ram_energy)

In [65]:
# Calculate emissions per 10,000 prompts
emissions_per_thousand_prompts = {
    'Total Emissions': total_emissions / num_prompts * 10_000,
    'CPU Energy': cpu_energy / num_prompts * 10_000,
    'GPU Energy': gpu_energy / num_prompts * 10_000,
    'RAM Energy': ram_energy / num_prompts * 10_000
}

In [66]:
print(emissions_per_thousand_prompts)

{'Total Emissions': array([ 0.71759476,  2.93206607,  7.22777361, 24.72467776]), 'CPU Energy': array([ 0.34119865,  1.38108   ,  3.41097504, 11.68430222]), 'GPU Energy': array([ 0.37330112,  1.55227741,  3.81306506, 13.01042801]), 'RAM Energy': array([ 0.36467404,  1.47610751,  3.64563793, 12.48812872])}


In [67]:
# Perform regression analysis
def perform_regression(x, y):
    x = x.reshape(-1, 1)
    model = LinearRegression()
    model.fit(x, y)
    predicted = model.predict(x)
    return model, predicted

In [68]:
models = {}
predictions = {}
for name, y in emissions_per_thousand_prompts.items():
    model, predicted = perform_regression(avg_output_tokens, y)
    models[name] = model
    predictions[name] = predicted
    print(f"{name} - Intercept: {model.intercept_}, Coefficient: {model.coef_[0]}")

Total Emissions - Intercept: -0.1372507836507495, Coefficient: 0.04224381387883653
CPU Energy - Intercept: -0.06708984508294691, Coefficient: 0.01996547599524041
GPU Energy - Intercept: -0.06762141160236013, Coefficient: 0.022225002718689008
RAM Energy - Intercept: -0.07169695534157139, Coefficient: 0.021338995289762592


In [69]:
# Prepare data for Altair visualization
vis_data = pd.DataFrame({
    'Parameters (billions)': parameters.flatten(),
    'Number of Examples': num_examples.flatten(),
    'Total Output Tokens': total_output_tokens.flatten(),
    'Total Input Tokens': total_input_tokens.flatten(),
    'Avg Input Tokens per Prompt': avg_input_tokens.flatten(),
    'Avg Output Tokens per Prompt': avg_output_tokens.flatten(),
    'Number of Prompts': num_prompts.flatten(),
    'Total Emissions per Ten-Thousand Prompts': emissions_per_thousand_prompts['Total Emissions'],
    'CPU Energy per Ten-Thousand Prompts': emissions_per_thousand_prompts['CPU Energy'],
    'GPU Energy per Ten-Thousand Prompts': emissions_per_thousand_prompts['GPU Energy'],
    'RAM Energy per Ten-Thousand Prompts': emissions_per_thousand_prompts['RAM Energy'],
    'Total Emissions Predicted': predictions['Total Emissions'],
    'CPU Energy Predicted': predictions['CPU Energy'],
    'GPU Energy Predicted': predictions['GPU Energy'],
    'RAM Energy Predicted': predictions['RAM Energy']
})

In [72]:
# Define chart width and height
chart_width = 600
chart_height = 350

# Create scatter and line plots for each emission type per million output tokens
charts = []
for emission_type, color in zip(['Total Emissions', 'CPU Energy', 'GPU Energy', 'RAM Energy'],
                                ['blue', 'green', 'red', 'purple']):
    scatter = alt.Chart(vis_data).mark_circle(size=100, color=color).encode(
        x='Avg Output Tokens per Prompt',
        y=f'{emission_type} per Ten-Thousand Prompts',
        tooltip=['Parameters (billions)', 
                 f'{emission_type} per Ten-Thousand Prompts', 
                 'Avg Output Tokens per Prompt', 
                 'Avg Input Tokens per Prompt', 
                 'Number of Examples', 
                 'Number of Prompts']
    ).properties(
        title=f'Actual {emission_type} per Ten-Thousand Prompts',
        width=chart_width,
        height=chart_height
    )
    
    # Create line plots for predicted emissions
    line = alt.Chart(vis_data).mark_line(color=color).encode(
        x='Avg Output Tokens per Prompt',
        y=f'{emission_type} Predicted',
        tooltip=['Parameters (billions)', 
                 f'{emission_type} per Ten-Thousand Prompts', 
                 'Avg Output Tokens per Prompt', 
                 'Avg Input Tokens per Prompt', 
                 'Number of Examples', 
                 'Number of Prompts']
    ).properties(
        width=chart_width,
        height=chart_height
    )
    
    charts.append(scatter + line)

# Arrange the charts in a 2x2 grid
grid_chart = alt.vconcat(
    alt.hconcat(charts[0], charts[1]),
    alt.hconcat(charts[2], charts[3])
).resolve_scale(
    y='independent'
)

grid_chart.show()

alt.VConcatChart(...)

In [75]:
# Create a combined chart with overlays for all emission types
combined_chart = alt.layer(*charts).resolve_scale(
    y='independent'
).properties(
    title='Emissions per Ten-Thousand Prompts by Type vs Avg Output Tokens',
    width=600,  # Adjusted width for combined chart
    height=400  # Adjusted height for combined chart
)

combined_chart.show()

alt.LayerChart(...)